In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import itertools as it
from sklearn.metrics import r

In [2]:
def generate_grid(params_grid):
        allNames = sorted(params_grid)
        combinations = it.product(
            *(params_grid[Name] for Name in allNames))
        dicts = []
        for item in list(combinations):
            temp = {}
            for key, value in zip(allNames, item):
                temp[key] = value
            dicts.append(temp)
        return dicts

In [3]:
space = {
             'n_estimators' : range(100, 1000, 400),
             'eta' : list(np.linspace(0.025, 0.5, 10)),
             'max_depth' : range(1,13),
             'min_child_weight' : list(range(1, 6)),
             'subsample' : list(np.linspace(0.5, 1, 3)),
             'gamma' : list(np.linspace(0.5, 1, 3)),
             'colsample_bytree' : list(np.linspace(0.5, 1, 3)),
             
             }

In [4]:
misc_params = {'eval_metric': 'rmse',
             'objective': 'reg:linear',
             'tree_method': 'exact',
             'silent' : 1}

In [5]:
combinations = generate_grid(space)

In [6]:
len(combinations)

48600

In [7]:
step = 4
split_range = range(9,34-step-1)
folds = []
for i in split_range:
    if i > min(split_range):
        temp_dict = {'train': [], 'valid': []}
        temp_dict['train'] = [min(split_range), i+step]
        temp_dict['valid'] = [i+1+step]
        folds.append(temp_dict)

In [8]:
optimization_task = {}
for comb in combinations:
    comb.update(misc_params)
    optimization_task[str(comb)] = []
    

In [ ]:
X = pd.read_csv('../input/X.csv', index=False)

In [ ]:
i = 0
for f in folds:
    train = f['train']
    valid = f['valid']
    X_train = X[X.date_block_num.isin(train)]
    X_cv = X[X.date_block_num == valid[0]]
    for param_set in combinations:
        param_set.update(misc_params)
        cols = [c for c in X_train.columns if c not in ['date_block_num', 'item_cnt_day']]
        x1 = X_train[cols]
        y1 = X_train['item_cnt_day']
        x2 = X_cv[cols]
        y2 = X_cv['item_cnt_day']
        watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
        model = xgb.train(params, xgb.DMatrix(x1, y1), 3500,  watchlist, maximize=False, verbose_eval=50, early_stopping_rounds=50)
        predictions = model.predict(xgb.DMatrix(X_cv[cols]), ntree_limit=model.best_ntree_limit)
        targets = y2
        rmse = np.sqrt(np.mean((predictions-targets)**2))
        optimization_task[str(param_set)].extend(rmse)
    i+=1
    print i
        

In [ ]:
combinations